## 使用bayesian optimization优化xgboost模型

本文实现的工作：
1. 调用Scikit-Learn API创建xgboost模型
2. 使用bayesian optimization 优化xgboost模型
3. 调用 imblearn.ensemble模块处理非平衡数据

In [1]:
import xgboost as xgb
import numpy as np 
import pandas as pd
import sklearn
from sklearn.cross_validation import KFold
from bayes_opt import BayesianOptimization

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# 评估方法

In [2]:
#计算rmse
#计算均方根误差RMSE
from __future__ import division
def calc_rmse1(forecast,real):
    #print 'forecast: ',forecast
    #print 'real: ',real.get_label()
    forecast1 = []
    for f in forecast:
        f = list(f)
        i = f.index(max(f))
        forecast1.append(i+1)
    diff = (forecast1-real.get_label())**2
    rmse = np.sqrt(diff.sum()/len(forecast1))
    #print 'rmse',rmse
    return rmse

In [3]:
#计算均方根误差RMSE
from __future__ import division
def calc_rmse(forecast,real):
    diff = (forecast-real)**2
    rmse = np.sqrt(diff.sum()/len(forecast))
    return rmse

### 自定义xgboost fit模型中的eval_metric参数

In [4]:
def eval_metric_func(y_predicted,y_true):
    rmse = calc_rmse1(y_predicted,y_true)
    name = 'mrmse'
    value = (name,rmse)
    return value

# 使用bayesian optimization优化模型

In [5]:
def xgbCv(train, features, eta, gamma, max_depth, min_child_weight, subsample, colSample,alpha,reg_lambda,scale_pos_weight):   
    # prepare xgb parameters 
    params = {
        'objective':'multi:softmax',
        "booster" : "gbtree",
        #"eval_metric": "mae",            
        #"tree_method": 'auto',
        "silent": 1,
        "learning_date": eta, 
        "max_depth": int(max_depth),
        "min_child_weight" : min_child_weight,
        "subsample": subsample, 
        "colsample_bytree": colSample,             
        "gamma": gamma,
        'reg_alpha':alpha,
        'reg_lambda':reg_lambda,
        'n_jobs':3,
        'n_estimators':100,
        'scale_pos_weight':scale_pos_weight
   }
    cvScore = kFoldValidation(train, features, params, nFolds = 3)
    print('CV score: {:.6f}'.format(cvScore)) 
    return -1.0 * cvScore   # invert the cv score to let bayopt maximize

In [6]:
  def bayesOpt(train, features):
    ranges = {            
        'eta': (0.001, 0.5),
        'gamma': (0.01, 25),
        'max_depth': (1, 10),
        'min_child_weight': (0.01, 10),
        'subsample': (0.01, 1),
        'colSample': (0.01, 1),
        'alpha':(0.01,1),
        'reg_lambda':(0.01,1),
        'scale_pos_weight':(0.1,10)
   }
    # proxy through a lambda to be able to pass train and features
    optFunc = lambda eta, gamma, max_depth, min_child_weight, subsample, colSample,alpha,reg_lambda,scale_pos_weight: xgbCv(train, features, eta, gamma, max_depth, min_child_weight, subsample, colSample,alpha,reg_lambda,scale_pos_weight)
    bo = BayesianOptimization(optFunc, ranges)
    bo.maximize(init_points = 50, n_iter = 5, kappa = 2, acq = "ei", xi = 0.0)
   
    bestMAE = round((-1.0 * bo.res['max']['max_val']), 6)
    print("\n Best MAE found: %f" % bestMAE)
    print("\n Parameters: %s" % bo.res['max']['max_params'])

In [7]:
def kFoldValidation(train, features, xgbParams, nFolds=5, target='score'):  
    kf = KFold(len(train), n_folds = nFolds, shuffle = True)
    fold_score=[]
    for train_index, cv_index in kf:
    # split train/validation
        X_train, X_valid = train[features].as_matrix()[train_index], train[features].as_matrix()[cv_index]
        y_train, y_valid = (train[target].as_matrix()[train_index]), (train[target].as_matrix()[cv_index])
        dtrain = xgb.DMatrix(X_train, y_train) 
        dvalid = xgb.DMatrix(X_valid, y_valid)
        
        clf = xgb.sklearn.XGBClassifier(**xgbParams)
        
        clf.fit(X_train, y_train,eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric=eval_metric_func, verbose=True)
        #alg.fit(train_data_features, train_labels)

        #Predict training set:
        dtrain_predictions = clf.predict(X_valid)
        rmse = calc_rmse(dtrain_predictions,y_valid)
        score = 1.0/(1+rmse)

        #score = sklearn.metrics.accuracy_score(y_train, dtrain_predictions)
        fold_score.append(score)                

    return np.mean(fold_score)

In [8]:
import pandas as pd
pd_train = pd.read_csv('./data/pd_train.csv')

In [9]:
feature = [str(i) for i in range(80)]

# 处理非平衡数据

In [10]:
df_group = pd_train.groupby('score')

In [11]:
#数据分布不平衡
for num,df in df_group:
    print num,df.shape

1 (434, 81)
2 (731, 81)
3 (7019, 81)
4 (21888, 81)
5 (44928, 81)


### EasyEnsemble模型

In [12]:
#使用imblearn.ensemble从非平衡数据集中抽取，构造平衡数据集
from imblearn.ensemble import EasyEnsemble
ee = EasyEnsemble(random_state=0,n_subsets=10)
X_resampled_ee,y_resampled_ee = ee.fit_sample(pd_train[feature],pd_train['score'])

### BalanceCascade模型

In [13]:
#balancecascade与emsemble不同之处在于使用classifier（parameter estimator）确保将错误分类的样本再次输入到下个子数据集中，使用n_max_subset设置子集的最大数量
#测试结果表明balancecascade效果比emsemble好一些
from imblearn.ensemble import BalanceCascade
from sklearn.linear_model import LogisticRegression
bc = BalanceCascade(random_state=0,estimator=LogisticRegression(random_state=0),n_max_subset=10)
X_resampled,y_resampled = bc.fit_sample(pd_train[feature],pd_train['score'])
print X_resampled.shape

(3, 2170, 80)


# 测试

### 基于subset训练BalancedBaggingClassifier算法模型

In [14]:
X_resampled1 = X_resampled[0]
y_resampled1 = y_resampled[0]
X_resampled2 = X_resampled[1]
y_resampled2 = y_resampled[1]

In [26]:
modi_X_resampled = X_resampled.reshape(2170*3,80)
pd_train1 = pd.DataFrame(modi_X_resampled,columns=feature)
modi_y_resampled = y_resampled.reshape(2170*3,)
pd_train1['score'] = modi_y_resampled

In [27]:
##结合resample和baggingClassifier
#测试结果表明xgboost模型比BalancedBaggingClassifier效果更好一些
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bbc = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                              ratio='auto',
                              replacement = False,
                              random_state=0)
#bbc.fit(pd_train[feature],pd_train['score'])
#bbc.fit(X_resampled1,y_resampled1)
bbc.fit(pd_train1[feature],pd_train1['score'])

BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
             bootstrap=True, bootstrap_features=False, max_features=1.0,
             max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=False,
             random_state=0, ratio='auto', replacement=False, verbose=0,
             warm_start=False)

In [31]:
y_pred1 = bbc.predict(X_resampled1)
calc_rmse(y_pred1,y_resampled1)

0.32697468158301396

In [47]:
#导入测试数据
import pandas as pd
pd_test = pd.read_csv('./data/pd_test.csv')
X_test = pd_test[feature]
y_test = pd_test['score']
#预测
y_pred = bbc.predict(X_test)
#评价
calc_rmse(y_pred,y_test)

pd_predict = pd.read_csv('./data/pd_predict.csv')

In [48]:
y_predict = bbc.predict(pd_predict[feature])

In [50]:
pd_predict['score_pre'] = y_predict

In [54]:
pd_predict.loc[:,['Id','score_pre']].to_csv('./data/bbc_test_result.csv',index=False)

### 基于subset训练xgboost模型

In [18]:
#构建xgboost模型
xgb1 = xgb.sklearn.XGBClassifier(
 learning_rate =0.4340921970827792,
 n_estimators=100,
 max_depth=9,
 min_child_weight=4.504951546403112,
 gamma=16.104447095482254,
 subsample=0.05273299539228559,
 colsample_bytree=0.45771700116167263,
 objective='multi:softmax',
 nthread=4,
 scale_pos_weight=7.095165216526744,
 reg_lambda=0.3909093700729328,
 reg_alpha=0.2574802155751231,
 seed=27)  


In [19]:
from sklearn.model_selection import GridSearchCV
def modelfit1(alg, train_data_features, train_labels,predict_data_features,useTrainCV=True, cv_folds=5):

    if useTrainCV:
        params=alg.get_xgb_params()
        xgb_param=dict([(key,[params[key]]) for key in params])

        boost = xgb.sklearn.XGBClassifier()
        cvresult = GridSearchCV(boost,xgb_param,cv=cv_folds)
        cvresult.fit(train_data_features, train_labels)
        alg=cvresult.best_estimator_


    #Fit the algorithm on the data
    alg.fit(train_data_features, train_labels)

    #Predict training set:
    dtrain_predictions = alg.predict(predict_data_features)
    #dtrain_predprob = alg.predict_proba(train_data_features)[:,1]
    return dtrain_predictions

In [20]:
y_pred2 = modelfit1(xgb1,X_resampled1,y_resampled1,X_resampled2)
calc_rmse(y_pred2,y_resampled2)

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

1.7336464327875656

### 基于subset优化xgboost模型

In [21]:
#从np.array构建pd.DataFrame,合并子数据集
modi_X_resampled = X_resampled.reshape(2170*3,80)
pd_train1 = pd.DataFrame(modi_X_resampled,columns=feature)
modi_y_resampled = y_resampled.reshape(2170*3,)
pd_train1['score'] = modi_y_resampled

In [22]:
pd_train1

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,score
0,-0.665478,0.751370,1.224749,-0.159357,0.496120,0.685247,1.766348,-0.864454,0.517406,1.293498,...,0.859138,4.404964,-1.250984,-0.475254,-1.707792,2.068623,-1.258309,-1.370042,-1.302643,2
1,-2.113851,-2.866650,3.000353,-1.194494,0.851301,2.281212,0.158847,0.989544,0.545823,1.798543,...,1.108199,4.197625,2.399921,0.460678,-3.284178,4.235124,0.650281,3.990605,-0.035036,2
2,-1.013846,-0.697878,1.850155,-2.183707,1.226869,3.716398,2.979673,2.344263,-0.296879,-1.612171,...,-0.101498,2.978921,0.317582,0.380673,-2.574484,1.914311,1.437153,-2.480369,-0.622769,2
3,6.162658,-0.437103,1.705616,3.862161,-1.543243,0.030413,0.447114,-6.672205,0.588504,-1.560186,...,-2.973648,4.465474,-0.863271,-3.087651,-6.084801,-5.132518,4.599639,2.661371,2.723480,2
4,0.063046,1.203075,2.993185,2.390523,0.539574,0.035865,2.358462,0.566969,-0.059333,0.527309,...,0.125946,1.188872,-0.045779,0.666349,-2.179602,2.012292,2.230582,-0.437695,-0.068362,2
5,-2.131821,-0.279158,0.789458,1.585624,0.686214,0.909586,1.417295,-0.545595,1.680117,1.331548,...,-0.132859,1.903035,-0.014279,0.092180,-2.055521,-0.040318,0.400534,0.439857,-0.880786,2
6,0.735321,-0.276945,2.810719,-0.208274,0.890326,-0.111231,1.222275,0.146300,-0.288393,0.785460,...,1.035937,1.133745,-0.173450,-0.348837,-1.113928,0.899855,-0.127299,0.909056,0.156056,2
7,-3.293263,-3.602768,1.384771,2.991270,-1.325762,1.025101,2.110281,-3.013374,-0.915585,-0.549241,...,2.200806,1.117575,0.795856,-0.354439,-0.973304,2.475048,2.268421,0.364577,-0.054241,2
8,1.009145,-1.882450,1.119720,-1.508792,1.467505,3.503239,2.580062,-2.063466,-0.102319,-0.842631,...,-0.351125,3.960012,2.283570,-0.391178,-1.906233,3.154448,-0.030779,-0.110296,-1.368085,2
9,0.687768,-1.870466,1.172369,1.193346,-0.220825,0.070686,1.625941,0.231475,0.699573,1.342060,...,1.093897,2.235161,-0.733015,1.833906,-0.791593,0.214344,1.521116,1.358417,0.683633,2


In [23]:
X_resampled0 = X_resampled[0]
y_resampled0 = y_resampled[0]
X_resampled1 = X_resampled[1]
y_resampled1 = y_resampled[1]
pd_train2 = pd.DataFrame(X_resampled0,columns=feature)
pd_train2['score'] = y_resampled0
pd_train2.head(5)

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,score
0,-0.665478,0.751370,1.224749,-0.159357,0.496120,0.685247,1.766348,-0.864454,0.517406,1.293498,...,0.859138,4.404964,-1.250984,-0.475254,-1.707792,2.068623,-1.258309,-1.370042,-1.302643,2
1,-2.113851,-2.866650,3.000353,-1.194494,0.851301,2.281212,0.158847,0.989544,0.545823,1.798543,...,1.108199,4.197625,2.399921,0.460678,-3.284178,4.235124,0.650281,3.990605,-0.035036,2
2,-1.013846,-0.697878,1.850155,-2.183707,1.226869,3.716398,2.979673,2.344263,-0.296879,-1.612171,...,-0.101498,2.978921,0.317582,0.380673,-2.574484,1.914311,1.437153,-2.480369,-0.622769,2
3,6.162658,-0.437103,1.705616,3.862161,-1.543243,0.030413,0.447114,-6.672205,0.588504,-1.560186,...,-2.973648,4.465474,-0.863271,-3.087651,-6.084801,-5.132518,4.599639,2.661371,2.723480,2
4,0.063046,1.203075,2.993185,2.390523,0.539574,0.035865,2.358462,0.566969,-0.059333,0.527309,...,0.125946,1.188872,-0.045779,0.666349,-2.179602,2.012292,2.230582,-0.437695,-0.068362,2


In [24]:
#1.盲目增加树木个数对实验效果改进不大
#2.数据平衡化后，效果更差？3/12
#优化
bayesOpt(pd_train2,feature)